In [ ]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())

def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())

def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions_map = {
    "get_ticker":get_ticker,
    "get_income_statement":get_income_statement,
    "get_balance_sheet":get_balance_sheet,
    "get_daily_stock_performance":get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4o-mini-2024-07-18",
#     tools=functions,
# )

assistant_id = "asst_7FDYhdQkkBLqoFpaBXHQyiXD"


In [33]:
thread = client.beta.threads.create(
    messages =[
        {
            "role":"user",
            "content": " I want to know if the Regetti Computing stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_N4xn7EBpgTCdBrKHOKeAQMAV', created_at=1742455598, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [34]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)

run

Run(id='run_UCaeeQzWEeR1SPXSC3Zt85Xo', assistant_id='asst_7FDYhdQkkBLqoFpaBXHQyiXD', cancelled_at=None, completed_at=None, created_at=1742455599, expires_at=1742456199, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_N4xn7EBpgTCdBrKHOKeAQMAV', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Fun

In [35]:
from sympy import content


def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling Function: {function.name} with arg {function.arguments}")
        outputs.append({
            "output": functions_map[function.name](json.load(function.arguments)),
            "tool_call_id": action_id,
            })
    return outputs


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role = "user",
        content=content        
    )

def get_messages (thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id,
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}:{message.content[0].text.value}")

In [36]:
send_message(thread.id,"please go ahead!")

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_N4xn7EBpgTCdBrKHOKeAQMAV while a run run_UCaeeQzWEeR1SPXSC3Zt85Xo is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [37]:
get_messages(thread.id)

user: I want to know if the Regetti Computing stock is a good buy


In [38]:
get_tool_outputs(run.id, thread.id)

Calling Function: get_ticker with arg {"company_name":"Regetti Computing"}


AttributeError: 'str' object has no attribute 'read'

In [39]:
get_run(run.id, thread.id).status


'requires_action'